In [1]:
import pandas as pd
df_oldversion = pd.read_csv("./train.csv")
df = pd.read_csv("./train_encodedv.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 185 entries, Id to SaleCondition_Partial
dtypes: bool(122), float64(3), int64(34), object(26)
memory usage: 892.7+ KB


In [2]:
df.columns[df.columns.str.startswith("Alley")]

Index(['Alley_Grvl', 'Alley_Pave'], dtype='object')

In [3]:
df_oldversion[df_oldversion["Alley"].notna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
21,22,45,RM,57.0,7449,Pave,Grvl,Reg,Bnk,AllPub,...,0,NaN,GdPrv,NaN,0,6,2007,WD,Normal,139400
30,31,70,C (all),50.0,8500,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2008,WD,Normal,40000
56,57,160,FV,24.0,2645,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2009,WD,Abnorml,172500
79,80,50,RM,60.0,10440,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2009,WD,Normal,110000
87,88,160,FV,40.0,3951,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2009,New,Partial,164500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1405,50,RL,60.0,10410,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,1,2006,WD,Family,105000
1414,1415,50,RL,64.0,13053,Pave,Pave,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,207000
1427,1428,50,RL,60.0,10930,Pave,Grvl,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Normal,140000
1432,1433,30,RL,60.0,10800,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,64500


In [4]:
# I erroneously one-hot encoded Alley column. Most rows NaN. It needs indic var, interaction term instead
# To correct it I will delete the on-hot encoded columns, then add original column back on
df.drop(["Alley_Grvl", "Alley_Pave"], axis=1, inplace=True)
df["Alley"] = df_oldversion["Alley"]

In [5]:
df_oldversion["Alley"].value_counts(), "" , df["Alley"].value_counts()

(Alley
 Grvl    50
 Pave    41
 Name: count, dtype: int64,
 '',
 Alley
 Grvl    50
 Pave    41
 Name: count, dtype: int64)

 # &#8659; Re-encoded LotShape: one-hot  &#8659;

In [6]:
df = pd.get_dummies(df, columns=["LotShape"])
"LotShape" in df.columns

False

In [7]:
df.columns[df.columns.str.startswith("LotShape")]

Index(['LotShape_IR1', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg'], dtype='object')

# &#8659; Happy that LandContour is encoded right &#8659;
 ## (Could also check correlation / r-squared with target variable to see which encoding method would be best)

In [8]:
df.columns[df.columns.str.startswith("LandContour")], "", df_oldversion["LandContour"].value_counts()

(Index(['LandContour_Bnk', 'LandContour_HLS', 'LandContour_Low',
        'LandContour_Lvl'],
       dtype='object'),
 '',
 LandContour
 Lvl    1311
 Bnk      63
 HLS      50
 Low      36
 Name: count, dtype: int64)

In [9]:
statement = f"""With LandContour one-hot encoded, Lvl should be True in 1311 columns. In reality it is true in {len(df.loc[df["LandContour_Lvl"] == True])} columns. 

Meanwhile, HLS should be True in 50 columns. In reality it is true in {len(df.loc[df["LandContour_HLS"] == True])} columns."""

print(statement)


With LandContour one-hot encoded, Lvl should be True in 1311 columns. In reality it is true in 1311 columns. 

Meanwhile, HLS should be True in 50 columns. In reality it is true in 50 columns.


# &#8659; One-hot Utilities &#8659;
## Would be great to delete this feature since all datapoints but one belong to single category. But we don't know how it is in test set

In [10]:
df["Utilities"].value_counts()

Utilities
AllPub    1459
NoSeWa       1
Name: count, dtype: int64

In [11]:
Utilities_order = {"AllPub": 0, "NoSewr": 1, "NoSeWa": 2, "ELO": 3}

In [12]:
df["Utilities_le"] = df["Utilities"].map(Utilities_order)
df["Utilities_le"]

0       0
1       0
2       0
3       0
4       0
       ..
1455    0
1456    0
1457    0
1458    0
1459    0
Name: Utilities_le, Length: 1460, dtype: int64

# &#8659; Happy with encoding of LotConfig &#8659;

In [13]:
df_oldversion["LotConfig"].value_counts()

LotConfig
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: count, dtype: int64

In [14]:
df.columns[df.columns.str.startswith("LotConfig")]

Index(['LotConfig_Corner', 'LotConfig_CulDSac', 'LotConfig_FR2',
       'LotConfig_FR3', 'LotConfig_Inside'],
      dtype='object')

# &#8659; LandSlope: complete &#8659;

In [15]:
df["LandSlope"].value_counts()

LandSlope
Gtl    1382
Mod      65
Sev      13
Name: count, dtype: int64

In [16]:
# To do this, label encode first
LandSlope_order = {"Gtl": 0, "Mod": 1, "Sev": 2}
df["LandSlope_le"] = df["LandSlope"].map(LandSlope_order)
df["LandSlope_le"].corr(df["SalePrice"])

np.float64(0.05115224817946653)

### &#8593; Insignificant correlation
#### so will label encode instead

In [17]:
df.drop("LandSlope_le", axis=1, inplace=True)

In [18]:
df = pd.get_dummies(df, columns=["LandSlope"])
df.columns[df.columns.str.startswith("LandSlope")]

Index(['LandSlope_Gtl', 'LandSlope_Mod', 'LandSlope_Sev'], dtype='object')

# &#8659; Happy with Neighborhood &#8659;   Could still compute r, though...

In [19]:
len(df.columns[df.columns.str.startswith("Neighborhood")])

25

In [20]:
len(df_oldversion["Neighborhood"].value_counts())

25

In [21]:
df.columns[df.columns.str.startswith("Neighborhood")]

Index(['Neighborhood_Blmngtn', 'Neighborhood_Blueste', 'Neighborhood_BrDale',
       'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
       'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown',
       'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW',
       'Neighborhood_Somerst', 'Neighborhood_StoneBr', 'Neighborhood_Timber',
       'Neighborhood_Veenker'],
      dtype='object')

## Condition1 and Condition2: combine them; dependent variables; (interaction terms)

## &#8659; BldgType: not done yet (did HouseStyle first) &#8659;  Compute r

In [22]:
df_oldversion["BldgType"].unique()

array(['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], dtype=object)

In [23]:
df.columns[df.columns.str.startswith("BldgType")]

Index(['BldgType_1Fam', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs',
       'BldgType_TwnhsE'],
      dtype='object')

In [24]:
df_oldversion.columns[df_oldversion.columns.str.startswith("BldgType")]

Index(['BldgType'], dtype='object')

## &#8659; HouseStyle: done &#8659;

In [25]:
df.columns[df.columns.str.startswith("HouseStyle")]

Index(['HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story',
       'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story',
       'HouseStyle_SFoyer', 'HouseStyle_SLvl'],
      dtype='object')

In [26]:
housestyle = df[['HouseStyle_1.5Fin', 'HouseStyle_1.5Unf', 'HouseStyle_1Story',
       'HouseStyle_2.5Fin', 'HouseStyle_2.5Unf', 'HouseStyle_2Story',
       'HouseStyle_SFoyer', 'HouseStyle_SLvl']]
df["HouseStyle"] = pd.from_dummies(housestyle)

In [27]:
df["HouseStyle"].value_counts()

HouseStyle
HouseStyle_1Story    726
HouseStyle_2Story    445
HouseStyle_1.5Fin    154
HouseStyle_SLvl       65
HouseStyle_SFoyer     37
HouseStyle_1.5Unf     14
HouseStyle_2.5Unf     11
HouseStyle_2.5Fin      8
Name: count, dtype: int64

In [63]:
df_oldversion["HouseStyle"].value_counts()

HouseStyle
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: count, dtype: int64

----------------------------------------------------------------------------------

#### &#8659; &#8659;              IMPORTANT: When I tried &#8659; &#8659; mapping the order to, then computing correlation on, df["HouseStyle"] -- the new DataFrame's version, it returned nan. And I checked for null values everywhere necessary. So I assume something about the undoing of one-hot encoding caused a problem. Because as you can see below, doing the same operations on the OLD DataFrame's column works. As does using LabelEncoder on the new DataFrame's column

#### In any case, the correlation is weak, so I will stick with one-hot encoding

In [28]:
housestyle_order = {"1Story": 0, "1.5Unf": 1, "1.5Fin": 2, "2Story": 3, 
                   "2.5Unf": 4, "2.5Fin": 5}
df_oldversion["HouseStyle_le"] = df_oldversion["HouseStyle"].map(housestyle_order)
df_oldversion["HouseStyle_le"].corr(df_oldversion["SalePrice"])

np.float64(0.14822486731584716)

In [29]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["HouseStyle_le"] = le.fit_transform(df["HouseStyle"])
df["HouseStyle_le"].corr(df["SalePrice"])

np.float64(0.18016262334399116)

In [30]:
df_oldversion.drop(columns="HouseStyle_le", inplace=True)

In [31]:
df.drop(["HouseStyle", "HouseStyle_le"], axis=1, inplace=True)
"HouseStyle" in df.columns

False

In [32]:
df.columns[df.columns.str.startswith("HouseStyle")].nunique()

8

In [33]:
df_oldversion.columns[df_oldversion.columns.str.startswith("HouseStyle")].nunique()

1

#### &#8659; Why so many null values in LotFrontage?? For diverse property types and lot configurations. How to fill in these nulls?

In [59]:
df["LotFrontage"].isna().sum()

np.int64(259)

In [61]:
df_oldversion.loc[df_oldversion["LotFrontage"].isna(), "LotConfig"].value_counts()

LotConfig
Inside     134
Corner      62
CulDSac     49
FR2         14
Name: count, dtype: int64

In [62]:
df_oldversion.loc[df_oldversion["LotFrontage"].isna(), "BldgType"].value_counts()

BldgType
1Fam      226
TwnhsE     22
Duplex      5
Twnhs       3
2fmCon      3
Name: count, dtype: int64